#  Leetcodeの復習

あえて言語化にこだわる。
とりあえず、新しく解いた問題を上に書き足していくが、もし関連する過去に解いた問があれば、それを浮上させてくる。


## [78. Subsets](https://leetcode.com/problems/subsets/)


### Sol1. Backtracking

自分で思いついた。    
- time = O(2**N * N) <- 最後の`*N`は、returnするリストにappendする時間
- space = O(2**N) <- returnするlistを除くと、curr_subsetのO(N)のみ。


```py

class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        self.nums = nums
        self.subsets = []
        self.curr_subset = []
        self.add_to_subsets(0)
        return self.subsets
        
    def add_to_subsets(self, idx):
        if idx == len(self.nums):
            self.subsets.append(self.curr_subset.copy())
            return
        
        # take current idx
        self.curr_subset.append(self.nums[idx])
        self.add_to_subsets(idx+1)
        self.curr_subset.pop()
        
        # don't take current idx
        self.add_to_subsets(idx+1)

```

### Sol2. Cascading

模範解答に書いてあった方法。そこまで素晴らしい方法なのかはなんとも言えん。と思ったが、spaceが定数で済むのはすごい。backtrackingのO(N)より全然すごい。

- time = O(2**N * N)
- space = O(1)


```py
class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        subsets = [[]]          ########### ここ空配列入れるの最初忘れていた。
        
        for num in nums:
            for i in range(len(subsets)):
                new_subset = subsets[i].copy()
                new_subset.append(num)
                subsets.append(new_subset)
        return subsets

```


### Sol3. Lexicographic (Binary Sorted) Subsets with Bitmask 

pythonだと、`bin(123) => '0b1111011'(文字列)`である。最初の2文字(0b)を取り除いてやる必要があるね。     
これを利用する。

- time O(2**N * N) <- bitmaskの作成, ret.appendのそれぞれでこれだけかかる。
- space O(2**N * N) <- bitmaskとretのそれぞれでこれだけかかる

```py

class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        ret = []

        # 最初これにしてたが、これだとprintの結果が、([2:]を抜くと)
        # ['0b0', '0b1', '0b10', '0b11', '0b100', '0b101', '0b110', '0b111'] になる。
        # つまり頭の0が入らなくなる。なるほど。
        # bitmasks = [bin(i)[2:] for i in range(2**len(nums))]
        # print(bitmasks)

        bitmasks = [bin(i)[3:] for i in range(2**len(nums), 2**len(nums) * 2)]  ### うーん天才。
        for bitmask in bitmasks:
            ret.append([num for i, num in enumerate(nums) if bitmask[i] == '1'])
        
        return ret
        
```

> It might seem simple at first glance to generate binary numbers, but the real problem here is how to deal with zero left padding, because one has to generate bitmasks of fixed length, i.e. 001 and not just 1. For that one could use standard bit manipulation trick:

なお、このやり方だと、元のnumsの要素が降順に並んでいるなら、要素がlexicographically sortedになる。
- e.g. `[3,2,1] > [[],[1],[2],[2,1],[3],[3,1],[3,2],[3,2,1]]`

(元が昇順だとこの逆、つまりlexicographically reversedになるわけではない!!!)
- e.g. `[1,2,3] > [[],[3],[2],[2,3],[1],[1,3],[1,2],[1,2,3]]`

### 所感

色々な方法があり勉強になった。sol2のcascadingの方がspaceが少なくて済むのは驚きだった。

また。sol3のbitmaskは最近お仕事の開発で出てきたので、ほーと思った。多分あれ書いた人この問題といたことあるな、、、

## [93. Restore IP Addresses](https://leetcode.com/problems/restore-ip-addresses/)



### Sol1. backtrackingで自分で解いた

以下の感じで、非常にややこしい解き方をしてた。まあ、解けたのはよかった。

- 末尾からtravereして、そのセクションの和が255を超えたらbreakする
- こうやって、ドットのインデックスを格納していき、self.retにappendする


### Sol2. backtrackingで模範解答を見てといた

模範解答より。

ー セクションの取り方は1−3桁のどれかなので、それでループを回して、取った数字がvalid化をみるのでよかった


```py
class Solution:
    def restoreIpAddresses(self, s: str) -> List[str]:
        self.s = s
        self.vals = []
        self.ret = []
        self.add_addresses(0)
        return self.ret
        
    def add_addresses(self, idx):
        vals_to_take = 4 - len(self.vals)             ######## 忘れたらTLEする。
        rem_digits = len(self.s) - idx
        if vals_to_take * 4 < rem_digits:
            return
        
        if len(self.s) == idx:
            if len(self.vals) == 4:
                self.ret.append('.'.join(self.vals))
            return
        if len(self.s) < idx:
            return
        
        for len_digits in (1,2,3):                     ######## この発想いい👍
            next_index = idx + len_digits
            if self.is_valid_address(idx, next_index-1):
                self.vals.append(self.s[idx:next_index])
                self.add_addresses(next_index)
                self.vals.pop()

    def is_valid_address(self, left, right):
        val = int(self.s[left:right+1])
        if self.s[left] == '0':                         ####### 忘れがち。0始まりなら、"0"しか許容されない。
            return left == right
        elif 1 <= val <= 255:
            return True
        return False
```


### 所感

backtrackingで解くというのは見当がついてた。
ただ、考えられるパターンは高々3 * 3 * 3 = 27通りでしかない、ということに気がついていなかったなあ。

## [333. Largest BST Subtree](https://leetcode.com/problems/largest-bst-subtree/)

binary treeが与えられる。このサブツリーの中でBSTとなっているもののうちで最大のサイズを求めよ。


### DFS with recursion

一点だけしくった。。   
以下のように、root.leftがBSTでなかったら確かにrootはBSTではない。しかし、root.rightはBSTかもしれないので、root.rightの探索は必要！！！

つまり、Falseを返してearly returnする前に、必ずroot.leftとroot.rightの探索を挟まないといけない。


```py
class Solution:
    def largestBSTSubtree(self, root: Optional[TreeNode]) -> int:
        self.largest_size = 0
        self.get_range_and_size(root)
        return self.largest_size
        
    def get_range_and_size(self, root):
        if root is None:
            return True, None, None, 0

        left_is_bst, left_min, left_max, left_size = self.get_range_and_size(root.left)
        #### ⏬ここに入れる!!!
        right_is_bst, right_min, right_max, right_size = self.get_range_and_size(root.right)
        if not left_is_bst:
            return False, None, None, None
        if left_max is not None and left_max >= root.val:
            return False, None, None, None
            
        #### right_is_bst, right_min, right_max, right_size = self.get_range_and_size(root.right)
        if not right_is_bst:
            return False, None, None, None
        if right_min is not None and root.val >= right_min:
            return False, None, None, None
        
        min_val = left_min or root.val
        max_val = right_max or root.val
        size = left_size + right_size + 1
        
        self.largest_size = max(self.largest_size, size)
        return True, min_val, max_val, size
```


### 所感

上記のしくりポイントだが、[298. Binary Tree Longest Consecutive Sequence](https://leetcode.com/problems/binary-tree-longest-consecutive-sequence/) のSol2(DFS with recursion)でも全く同じ引っかかり方をした。DFS with recursion(つまりtop down)の重要な注意点なのかもしれない。

## [163. Missing Ranges](https://leetcode.com/problems/missing-ranges/)



### Sol1. Liner Scan


バッチリ思いついた。「startとendを受け取り、その範囲を文字列化して返すリストに詰める」という処理を`def add_to_arr`を切り出すところが肝。これは、[68. Text Justification](https://leetcode.com/problems/text-justification/) を前日に解いていたので瞬時に思いついた。

```py
class Solution:
    def findMissingRanges(self, nums: List[int], lower: int, upper: int) -> List[str]:
        self.ret = []
     
        prev = lower 
        for num in nums:
            self.add_to_arr(prev, num - 1)
            prev = num + 1
        self.add_to_arr(prev, upper)
        return self.ret       
            
    def add_to_arr(self, start, end):
        if start > end:
            return
        if start == end:
            self.ret.append(str(start))
        else:
            self.ret.append("{}->{}".format(start, end))
```

### 所感

ちゃんと問題を読んでおらず、numsにはlower~upperの範囲外の数字も含まれてしまうと勘違いして最初にbinary searchを咬ましてしまってた。答えは変わらないものの、、ちゃんと問題読もう。

## [1477. Find Two Non-overlapping Sub-arrays Each With Target Sum](https://leetcode.com/problems/find-two-non-overlapping-sub-arrays-each-with-target-sum)


数列が与えられる。この中で、「和がtargetとなるsubarray」をかぶる部分がないように二つ取る。二つの配列の長さの和の最小値を求めよ。そのような組み合わせが存在しない場合には−1を返せ。


### Sol1. Prefix Sum + bruite force to find a best pair = O(N**2) time



```py
class Solution:
    def minSumOfLengths(self, arr: List[int], target: int) -> int:
        cum_sum = 0
        # 全ての要素は正なのでcum_sum: idxは一対一に対応する。
        # 要素に0が含まれるときは、listで持っておく必要があるかも。
        # (idxが必要ない場合はcntを持っておくのでいいこともある)
        cum_sum_to_idx = {0: -1}
        segments = []
        
        for i, num in enumerate(arr) :
            cum_sum += num                                       # prefix sum の更新
            if (cum_sum - target) in cum_sum_to_idx:
                last_idx = cum_sum_to_idx[cum_sum - target]
                segments.append((last_idx + 1, i))
            cum_sum_to_idx[cum_sum] = i                          # hashmapに、このsumをキーとして追加
        
        if len(segments) < 2:
            return -1

        ret = -1
        for i in range(len(segments)):                           # 2つのsubarrayを総当たりしてかぶってないか調べる
            for j in range(i+1, len(segments)):
                seg1, seg2 = segments[i], segments[j]
                if seg1[1] < seg2[0] or seg2[1] < seg1[0]:
                    # print(seg1, seg2)
                    sum_segment = self.len_segment(seg1) + self.len_segment(seg2)
                    ret = sum_segment if ret == -1 else min(ret, sum_segment)
        return ret
        

```

### So2. Prefix Sum + create 2 lists to store min length in [left|right] sides = O(N) time

(c.f. [238. Product of Array Except Self](https://leetcode.com/problems/product-of-array-except-self/) の`Approach 1: Left and Right product lists`)

配列を二つ要素する。一つ目はインデックスiの要素には(arrのidx<iより前のスコープで条件を満たすsubarrayの長さの最小値)が収納される。二つめには、idx=iの要素には(arrのi<=idxのスコープで条件を満たすsubarrayの長さの最小値)が収納される。これらを同時にzipしてtraverseし、和を求めていき、その最小値を返す。という発想。   


```py

### 後半部のみ実装

        # min length of subarray which is in the range of (0 ~ idx-1)
        min_length_in_left = [None] * len(arr)
        next_idx = 0
        for i in range(1, len(arr)):
            
            if segments[next_idx][1] == i - 1:
                if min_length_in_left[i-1] is not None:
                    min_length_in_left[i] = min(
                        min_length_in_left[i-1],
                        segments[next_idx][1] - segments[next_idx][0] + 1,
                    )
                else:
                    min_length_in_left[i] = segments[next_idx][1] - segments[next_idx][0] + 1
                if next_idx + 1 <= len(segments)-1:
                    next_idx += 1
            else:
                min_length_in_left[i] = min_length_in_left[i-1]
        
        # min length of subarray which is in the range of (idx ~ end)
        min_length_in_right = [None] * (len(arr) + 1) # last element is placeholder
        next_idx = len(segments)-1
        for i in range(len(arr)-1, -1, -1):
            if segments[next_idx][0] == i:
                if min_length_in_right[i+1] is not None:
                    min_length_in_right[i] = min(
                        min_length_in_right[i+1],
                        segments[next_idx][1] - segments[next_idx][0] + 1,
                    )
                else:
                    min_length_in_right[i] = segments[next_idx][1] - segments[next_idx][0] + 1
                if 0 <= next_idx - 1:
                    next_idx -= 1
                
            else:
                min_length_in_right[i] = min_length_in_right[i+1]

        ret = None
        for left_sum, right_sum in zip(min_length_in_left, min_length_in_right[:-1]):
            if left_sum is None or right_sum is None:
                continue
            if ret is not None:
                ret = min(ret, left_sum + right_sum)
            else:
                ret = left_sum + right_sum
                
        return ret if ret is not None else -1
        
```

これだと(segmentsとlen(arr)の配列の二つを同時にるーぷを回さないといけないので)実装が大変。    
https://leetcode.com/problems/find-two-non-overlapping-sub-arrays-each-with-target-sum/discuss/725446/C%2B%2B-Clean-2-pass-solution を参考に以下の実装をした。多少綺麗にはなったのだろうか、、、、？)



```py
class Solution:
    def minSumOfLengths(self, arr: List[int], target: int) -> int:
        cum_sum = 0
        cum_sum_to_idx = {0: -1}
        idx_to_cum_sum = {-1: 0}
        
        
        for i, num in enumerate(arr) :
            cum_sum += num
            cum_sum_to_idx[cum_sum] = i
            idx_to_cum_sum[i] = cum_sum
        
        
        # min length of subarray which is in the range of [0 ~ idx-1]
        curr_min = None
        min_length_in_left = [None]
        for end in range(0, len(arr)-1):
            if idx_to_cum_sum[end] - target in cum_sum_to_idx:
                start = cum_sum_to_idx[idx_to_cum_sum[end] - target] + 1
                if (curr_min is None) or (end - start + 1 < curr_min):
                    curr_min = end - start + 1
            min_length_in_left.append(curr_min)
            
        curr_min = None
        min_length_in_right = [None] * len(arr)
        for start in range(len(arr)-1, -1, -1):
            if idx_to_cum_sum[start-1] + target in cum_sum_to_idx:
                end = cum_sum_to_idx[idx_to_cum_sum[start-1] + target]
                if (curr_min is None) or (end - start + 1 < curr_min):
                    curr_min = end - start + 1
            min_length_in_right[start]  = curr_min
            
            
        ret = None
        for left_sum, right_sum in zip(min_length_in_left, min_length_in_right[:-1]):
            if left_sum is None or right_sum is None:
                continue
            if ret is not None:
                ret = min(ret, left_sum + right_sum)
            else:
                ret = left_sum + right_sum
                
        return ret if ret is not None else -1

```

### 所感

(cumulative sumと僕が呼んでいたものは、prefix sumとも言うらしい)    

「前半の和がkになるsubarrayを求める」箇所に関しては、[560. Subarray Sum Equals K](https://leetcode.com/problems/subarray-sum-equals-k/)でやった通り、「Cumulative Sum + hashmap」を使う。(できた)    


ここからが自分で思いつかなかった。貪欲に区間長が短くてセグメントを貪欲に取るのではダメそうということはわかっていた。(e.g. [-100,0], [1,3],[3,4],[4,6],[7,100]の時、まず[3,4]を取ったら最適解が見つからなくなる。)

そこで、O(N**2)時間かかる総当たりをしたがTLE。ここで、答えを見た。[これ](https://leetcode.com/problems/find-two-non-overlapping-sub-arrays-each-with-target-sum/discuss/685486/JAVA-O(N)-Time-Two-Pass-Solution-using-HashMap.)を見て、ある要素より左側での最小の長さ、右側で最小の長さを保持する配列を二つ用意し、それをtraverseすればO(N)でできることを学んだ。

これは、[238. Product of Array Except Self](https://leetcode.com/problems/product-of-array-except-self/) の`Approach 1: Left and Right product lists`と同じような方法。うーん賢い。

## [298. Binary Tree Longest Consecutive Sequence](https://leetcode.com/problems/binary-tree-longest-consecutive-sequence/)

binary treeが与えられる。子が親のval + 1になっている時それらを連続していると呼ぶことにする。連続する最長のパスの長さを返せ。

### Sol1. DFS with Stack

これはいけた。

### Sol2. DFS with recursion (bottom up)

これに思いのほかハマった。特に以下の三点。

```py
class Solution:
    def longestConsecutive(self, root: Optional[TreeNode]) -> int:
        self.ret = 0
        self.get_curr_concutive_len(root)
        return self.ret
        
    def get_curr_concutive_len(self, root):   ########### (1)現在の「連続する」長さを返している。(子以下のmax長ではなく)
        ret = 1 # root cannot be None here.
        if root.left is not None:
            left_conc_length = self.get_curr_concutive_len(root.left)
            if root.left.val == root.val + 1:
                ret = max(ret, left_conc_length + 1)  ### (2) root.val = root.left.val+1 でなくても子の探索は必要。
        if root.right is not None:
            right_conc_length = self.get_curr_concutive_len(root.right)
            if root.right.val == root.val + 1:       
                ret = max(ret, right_conc_length + 1) ### (2)同上。
        self.ret = max(self.ret, ret)         ########### (3)ここで、tree中の「歴代の」最長を更新している。
        return ret
```


(2)は、[333. Largest BST Subtree](https://leetcode.com/problems/largest-bst-subtree/) でも同様の引っかかり方をした。

### Sol3. DFS with recursion (top down)

これはいけた。

```py
class Solution:
    def longestConsecutive(self, root: Optional[TreeNode]) -> int:
        self.longest = 1
        self.update_longest(root.left, 1, root.val)
        self.update_longest(root.right, 1, root.val)
        return self.longest
    
    def update_longest(self, root, curr_len, par_val):
        if root is None:
            return
        if par_val + 1 == root.val:
            curr_len += 1
        else:
            curr_len = 1
        self.longest = max(self.longest, curr_len)
        
        for child in [root.left, root.right]:
            self.update_longest(child, curr_len, root.val)
```


### 所感

sol1(=stack), sol3のようにtop-downだと簡単にできた。が、sol2のようにbottop upだと(僕にとっては)少しややこしかった。


## [1352. Product of the Last K Numbers](https://leetcode.com/problems/product-of-the-last-k-numbers/)

以下の二つのくえりの結果を返すシステムを実装せよ。
- add (0以上の整数を加える)
- getProduct(k) (最後に加えたk個の数字の積を掛け合わせる)


## Sol1. Cumulative Product + 0 indexes array

```py
class ProductOfNumbers:
    def __init__(self):
        self.products = [1] # note 1 as the first element
        self.zero_idxes = []
        
    def add(self, num: int) -> None:
        if num == 0:
            self.zero_idxes.append(len(self.products))
            num = 1
        self.products.append(self.products[-1] * num)

    def getProduct(self, k: int) -> int:
        # check if 0 exists in idx=len(self.products)-k ~ len(self.products)-1
        # ⏫非常にややこしいが、最後に入れた数字は len(self.products)-1 番目に入れた数字だということを意識しよう。
        idx = bisect.bisect_left(self.zero_idxes, len(self.products)-k)
        if idx < len(self.zero_idxes) and self.zero_idxes[idx] <= len(self.products)-1:
            return 0
        return self.products[-1] // self.products[-1-k]
```


## Sol 2. Cumulative Product with resetting every time 0 appears.

Solution1の改善版。

> If you have a '0' in input... reset the array since product upto that would be zero.

この天才的なアイデアを使用している。

```py

class ProductOfNumbers:
    def __init__(self):
        self.products = [1]
    def add(self, num: int) -> None:
        if num == 0:
            self.products = [1]
        else:
            self.products.append(self.products[-1] * num)
            
    def getProduct(self, k: int) -> int:
        if k > len(self.products) - 1:
            return 0
        else:
            return self.products[-1] // self.products[-1-k]

```

https://leetcode.com/problems/product-of-the-last-k-numbers/discuss/867055/Faster-than-100-simple-logic より。

### 所感

Sol1では、idxでちょっと、いやかなり混乱した。Sol2は天才的なアイデアで感動してしまった。

## [46. Permutations](https://leetcode.com/problems/permutations/)



異なる要素からなる数列が渡される。これを並び替えて作られる全ての配列を要素としてもつ配列を返せ。


### Sol 1. backtracking (参照対象の配列が更新されるバグがあった) 

(知見) **backtrackingは、「引いたり戻したりしてる集合(list, set,,,)」はbacktrackingの各過程でmutableに書き換えられるので、これのスナップショットを撮っておくには、#copy(deep copy) したものを保存していかないといけない。**

```py
class Solution:
    def permute(self, nums: List[int]) -> List[List[int]]:
        permutated_nums = []
        rem_nums = set(nums)
        
        def add_permutated_lists():
            if len(rem_nums) == 0:
                # (1) そして何より、retに入っているpermutated_numsは配列の参照なので、backtracking の過程で書き換えられる。
                ret.append(permutated_nums) # ret.append(permutated_nums.copy())にしたらOK。
                return
            # (2) ループを回しているrem_numsをループの中で書き換えてしまっている。
            for num in rem_nums:             # for num in list(rem_nums)、か、rem_nums[:] なら以下の理由によりOK。ただし、これだと、結局各スタックで配列を持っておく必要があるので、backtrackingによるspaceの節約ができてない気がする。
                permutated_nums.append(num)
                rem_nums.remove(num)
                add_permutated_lists()
                del permutated_nums[-1]
                rem_nums.add(num)
        ret = []
        add_permutated_lists()
        return ret

# 結果は [[],[],[],[],[],[]]

# --- 以下より、list(some_set) は、要素をそれぞれコピーしてるっぽいので、(2)はlist(xxx)にすればOKだとわかる--
>>> x = {1,2,3}
>>> y = list(x)
>>> x.remove(1)
>>> y
[1, 2, 3]
```


## [68. Text Justification](https://leetcode.com/problems/text-justification/)

単語の配列が渡される。それを、各行がmaxWidth以下の長さになるように、単語間の空白空白スペースを均等にして、left justified かつ right justifiedにして、各行の文字列を各要素とする配列にして返せ。


ただし、
- その行に一単語しかない場合は、左づめ。
- 最後の行は左詰


### Sol1. ループ

自分で思いついた。同じ行に入る単語たちのstart_idx, end_idxを受け取って、ret(配列)に詰めるという処理を関数に切り出すことで、シンプルに実装できた。    

[163. Missing Ranges](https://leetcode.com/problems/missing-ranges/) で


### 所感

hardというかややこしいだけ。これをつまらずにできるようになったのは、レベルアップした証拠では。  
この問題を解いた翌日に、[163. Missing Ranges](https://leetcode.com/problems/missing-ranges/) をついた。これも「区間を決めるまでメイン関数でやって、それをヘルパー関数に渡してそこで処理する」ようにしており、シュッと思いついた。


## [221. Maximal Square](https://leetcode.com/problems/maximal-square/)

m x n の(0 or 1)の長方形が与えられる。1のみからなる正方形の面積の最大値を求めよ。


### Sol1. DP with 2d space

```py
class Solution:
    def maximalSquare(self, matrix: List[List[str]]) -> int:
        dp = [[None] * len(matrix[0]) for _ in range(len(matrix))]
        dp[0] = matrix[0].copy()
        # ret = max(dp[0]) # 0 or 1 currently
        
        for row in range(1, len(matrix)):
            dp[row][0] = matrix[row][0]
            # ret = max(ret, dp[row][col])
            
            for col in range(1, len(matrix[0])):
                dp[row][col] = min(
                    dp[row-1][col-1],
                    dp[row-1][col],
                    dp[row][col-1]
                ) + 1
        max_side_len = max([max(row) for row in col])
        return max_side_len ** 2
```


### Sol2. DP with 1d space

1dといいつつ、二行分スペースを使うことにより、実装がシンプルになる。

```py
class Solution:
    def maximalSquare(self, matrix: List[List[str]]) -> int:
        dp = matrix[0]
        max_side_len = max(dp) # 0 or 1
        
        for row in matrix[1:]:
            new_dp = [row[0]]                ########## ここ
            for col_idx in range(1, len(matrix[0])):
                new_dp.append(
                    min(dp[col_idx-1], dp[col_idx], new_dp[col_idx-1]) + 1
                )
            max_side_len = max(max(new_dp), max_side_len)
            dp = new_dp
        return max_side_len ** 2
```



### 所感

2021/5に一度め解いていたく感動した問題。2021/8に2回めを解いたが、記憶に媚びいついて残っていた。

## [1293. Shortest Path in a Grid with Obstacles Elimination]()

二次元grid(0=road, 1=obstacle)を考える。 最大k個までブロックを壊していいとき、(0,0) -> (len(gird)-1, len(grid[0])-1) までの最短経路を求めよ。


### Sol1. BFS

最初、{各ステップ: 要素の配列}をハッシュで持って、しかも「ダイクストラ法の応用」と悦に浸っていたが、よくよく考えるとただBFS。

```py
class Solution:
    def shortestPath(self, grid: List[List[int]], k: int) -> int:
        target = (len(grid)-1, len(grid[0])-1)
        if target == (0, 0):
            return 0
        q = deque([(k, (0,0))]) # (num_blockables, cell)
        seen = {(0, 0): k}
        curr_step = 0
        
        while 0 < len(q):
            curr_step += 1
            for _ in range(len(q)):
                num_b, cell = q.popleft()

                for ver, hor in [(-1,0), (0,1), (1,0), (0,-1)]:
                    next_cell = (cell[0]+ver, cell[1]+hor)

                    if next_cell[0] < 0 or len(grid) <= next_cell[0]:
                        continue
                    if next_cell[1] < 0 or len(grid[0]) <= next_cell[1]:
                        continue
                    if next_cell == target:
                        return curr_step

                    next_num_blockables = num_b - grid[next_cell[0]][next_cell[1]]

                    if next_num_blockables < 0:
                        continue
                    
                    #### ひねりがあるとしたらここだけ
                    if next_cell in seen and next_num_blockables <= seen[next_cell]:
                        continue
                    q.append((
                        next_num_blockables,
                        next_cell
                    ))
                    seen[next_cell] = next_num_blockables
        return -1
                    
```

### Sol 2. A* メソッド

templates/202126_heuristicsを参照のこと。

```py
class Solution:
    def shortestPath(self, grid: List[List[int]], k: int) -> int:
        self.grid = grid
        target = (len(grid)-1, len(grid[0])-1)
        if target == (0, 0):
            return 0
        
        if k >= self.manhattan((0,0)):
            return self.manhattan((0,0))
        
        seen = set([(k, (0, 0))]) # (num_blackables, cell)
        h = [(self.manhattan((0, 0)), 0, k, (0, 0))] # f(n), steps_so_far, num_blackables, cell
        while len(h) > 0:
            _, steps, num_b, cell = heapq.heappop(h)
            
            if cell == target:               ####### 多分ここで返さないといけない!!!!!!!!!
                return steps
            
            

            for ver, hor in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                next_cell = cell[0] + ver, cell[1] + hor

                if next_cell[0] < 0 or len(grid) <= next_cell[0]:
                    continue
                if next_cell[1] < 0 or len(grid[0]) <= next_cell[1]:
                    continue
                
                
                　                           #### 同じ階層のnodeは一応全部見ないといけない!!!!!
#                 if target == next_cell:    #### 多分ここで返してはいけない!!!!!!!!!
#                     return steps + 1
                next_num_b = num_b - grid[next_cell[0]][next_cell[1]]
                if next_num_b < 0:
                    continue
                if (next_num_b, next_cell) in seen:
                    continue
                
                if self.manhattan(next_cell) <= next_num_b:
                    return steps + 1 + self.manhattan(next_cell)
                
                heapq.heappush(h, (steps+1 + self.manhattan(next_cell), steps+1, next_num_b, next_cell))
                seen.add((next_num_b, next_cell))
        return -1
    
    # ヒューリスティック関数として使っている。
    def manhattan(self, cell):
        return len(self.grid)-1 - cell[0] + len(self.grid[0])-1 - cell[1]
```        
        
        
### 所感

A*メソッドについて学んだ。「ヒューリスティック関数がadmissiveだったら、ちゃんと最適な経路が返される」ことはしっくりきてない。

## [5. Longest Palindromic Substring](https://leetcode.com/problems/longest-palindromic-substring/)


最長のpalindrome substringを返せ。


### Sol1. 2 pointers

- traverseの処理をhelperメソッドに切り出すことで、実装がシンプルになる。むしろ、切り出さないと地獄。

```py
class Solution:
    def longestPalindrome(self, s: str) -> str:
        self.s = s
        self.ret = ''
        for left in range(len(s)):
            self.update_ret(left, left)
            if left + 1 < len(s):
                self.update_ret(left, left+1)
        return self.ret
            
    def update_ret(self, left, right):
        while 0 <= left-1 and right+1 <= len(self.s)-1 and self.s[left-1]==self.s[right+1]:
            left -= 1
            right += 1
        if len(self.ret) < right - left + 1:
            self.ret = self.s[left:right+1] 
```


### Sol2. DP

todo
spaceもtimeもO(n**2)かかるので、旨味はそんなにない。



### 所感

エッジケース忘れがち。

## [490. The Maze](https://leetcode.com/problems/the-maze/)




### Sol 1. DFS

普通のDFS。edge caseの`start == destination`だけ見逃さないように。    
Assuming that maze size is n * m,   

- time: O(nm) 
    - when we traverse all the cells
- space: O(nm)
    - length of the stack(max nm) + visited(max nm)
    
    
### Sol 2. BFS

まあ、これBFSで解こうとは思わんけどw



### 所感
問題ちゃんと読んでなくて時間溶かした。DFS使うだけ。鬱。



## [743. Network Delay Time](https://leetcode.com/problems/network-delay-time/)





### 所感




前問1976が、dijkstra法の問題だったので、復習。    
See: https://github.com/kudojp/RoadToSuperCoder/blob/master/algorithms/templates/210418_Dijkstra.ipynb

## 1976. Number of Ways to Arrive at Destination

N個のnodeからなる、重み付きのBidirected Grpahが与えられる。これはNode(0)から、Node(n-1)に最小経路でたどり着く経路は何種類あるか？


### Sol1. TLE. backtracking

210821のbiweeklyで出題された。最初はbacktrackingで実装して、TLEした。    
またこれに「最短経路よりも大きいことが判明した時点でearly returnする」という追加実装を入れたが、それでもTLE。

    
```py
class Solution:
    def countPaths(self, n: int, roads: List[List[int]]) -> int:
        self.n = n
        self.adj_lists = defaultdict(list)
        for road in roads:
            self.adj_lists[road[0]].append((road[1], road[2]))
            self.adj_lists[road[1]].append((road[0], road[2]))

        self.curr_cost = 0
        self.visiteds = set([0])
        self.min_cost, self.num_shortest_ways = 300 * 200, 0
        self.travel_from(0)
        return self.num_shortest_ways % (10**9 + 7)
        
        
    def travel_from(self, city):
        # print(city, self.visiteds, self.curr_cost)
        if city == self.n-1:
            # print(self.curr_cost, self.min_cost)
            if self.curr_cost < self.min_cost:
                self.min_cost, self.num_shortest_ways = self.curr_cost, 1
            elif self.curr_cost == self.min_cost:
                self.num_shortest_ways += 1
            return
            
        for next_city, cost in self.adj_lists[city]:
            if next_city in self.visiteds:
                continue
            
            self.visiteds.add(next_city)
            self.curr_cost += cost
            self.travel_from(next_city)
            self.visiteds.remove(next_city)
            self.curr_cost -= cost

```



### Sol2. TLE. dikjstra


その後、色々考えて、dijkstraで実装できそうだということを思いついた。が、heapの中に全ての要素を突っ込む以下の実装だとTLEした。基本思想としては、「dijkstraで、heapから要素(距離、node#)を取り出す時に、同じ距離の要素はまとめて取り出し、そこで初めて取り出したnode#に関しては、その距離が最短経路なので、node#の個数を数えたらそれが最短経路になる」というもの。

```py

class Solution:
    def countPaths(self, n: int, roads: List[List[int]]) -> int:
        self.adj_lists = defaultdict(list)
        for road in roads:
            self.adj_lists[road[0]].append((road[1], road[2]))
            self.adj_lists[road[1]].append((road[0], road[2]))
        
        marked = [(0, 0, 1)] # min heap (dis from root, city id, # from routes)
        temp_fixed = defaultdict(int)
        fixed = defaultdict(int)
        
        while len(fixed) < n:
            # print(fixed, marked)
            dist_to_be_fixed = marked[0]
            while 0 < len(marked) and marked[0] == dist_to_be_fixed:            
                dist, city, cnt = heapq.heappop(marked)
                if city in fixed:
                    continue

                for adj_city, dist_road in self.adj_lists[city]:
                    if adj_city not in fixed:
                        heapq.heappush(marked, (dist + dist_road, adj_city, cnt))
            
                temp_fixed[city] += cnt
            for city, cnt in temp_fixed.items():
                fixed[city] += cnt
            temp_fixed = defaultdict(int)
            
            if n-1 in fixed:
                return fixed[n-1] % (10**9 + 7)


```


### Sol3. Acc. dijkstra + hashmap of (dist, cnt) in heap

そこで、dijkstraのheapの中の要素を、cities_in_hで持っておくという以下の実装により、Acceptedになった。


```py
from collections import defaultdict
import heapq

class Solution:
    def countPaths(self, n: int, roads: List[List[int]]) -> int:
        self.adj_lists = defaultdict(list)
        for road in roads:
            self.adj_lists[road[0]].append((road[1], road[2]))
            self.adj_lists[road[1]].append((road[0], road[2]))
        
        h = [(0, 0)] # min heap (dis from root, city id)
        cities_in_heap = {0: (0, 1)}   # city to (min dist from root, cnt)
        fixed = defaultdict(int)
        
        while len(fixed) < n:   
            cost, city = heapq.heappop(h)
            _, cnt = cities_in_heap[city]

            if city == n - 1:
                return cnt % (10**9 + 7)
            
            if city in fixed:
                continue

            for adj_city, adj_dist in self.adj_lists[city]:
                if adj_city in fixed:
                    continue

                if adj_city in cities_in_heap:
                    cost_in_heap, cnt_in_heap = cities_in_heap[adj_city]
                    if cost + adj_dist < cost_in_heap:
                        cities_in_heap[adj_city] = (cost + adj_dist, cnt)
                    elif cost + adj_dist == cost_in_heap:
                        cities_in_heap[adj_city] = (cost_in_heap, cnt_in_heap + cnt)
                    continue

                heapq.heappush(h, (cost + adj_dist, adj_city))
                cities_in_heap[adj_city] = (cost + adj_dist, cnt)
                
            fixed[city] = cities_in_heap[city][1]
            del cities_in_heap[city]
```

### Sol4. (答えみた) dijkstra + 2 hashmaps of dist and of cnt

https://leetcode.com/problems/number-of-ways-to-arrive-at-destination/discuss/1417573/python-Almost-Dijktra-explained


Sol3とやってることは同じ?だが、圧倒的に読みやすい。

```py
class Solution:
    def countPaths(self, n: int, roads: List[List[int]]) -> int:
        adj_list = defaultdict(list)
        for x, y, w in roads:
            adj_list[x].append((y, w))
            adj_list[y].append((x, w))
        
        cnts = defaultdict(int)
        cnts[0] = 1
        dists = { i: 200**300 for i in range(n) }
        dists[0] = 0
        h = [(0, 0)] # (distance from root, node#)
        
        while 0 < len(h):
            curr_dist, curr_node = heapq.heappop(h)
            if dists[curr_node] < curr_dist:                         ### heapの中のそnodeの最小distではない場合
                continue

            for adj_node, adj_dist in adj_list[curr_node]:
                if dists[adj_node] == curr_dist + adj_dist:
                    cnts[adj_node] += cnts[curr_node]
                    # heappush(h, (curr_dist + adj_dist, adj_node))  ### == の時はheapに突っ込まない
                elif dists[adj_node] > curr_dist + adj_dist:
                    cnts[adj_node] = cnts[curr_node]
                    dists[adj_node] = dists[curr_node] + adj_dist
                    heappush(h, (curr_dist + adj_dist, adj_node))    ### 今のheapの中のdistより小さい場合のみ突っ込む
        return cnts[n-1] % (10**9 + 7)
```


### 所感


なかなかむずいな。sol4は、見事に自分がsol3で汗まみれでやっていたことをシンプルに実現してくれて天才的だと思った。




演算子, parenthesis関係のstackの問題をざっと見返してみる。(あげれるだけあげた。全部あげれたわけではない。)

## 20. https://leetcode.com/problems/valid-parentheses/

## 22. https://leetcode.com/problems/generate-parentheses/

## 1249. https://leetcode.com/problems/minimum-remove-to-make-valid-parentheses/submissions/


復習TODO

## [1003. Check If Word Is Valid After Substitutions](https://leetcode.com/problems/check-if-word-is-valid-after-substitutions/)

来週TODO


## [772. Basic Calculator III](https://leetcode.com/problems/basic-calculator-iii/)

来週TODO

## 224. [Basic Calculator ](https://leetcode.com/problems/basic-calculator/)

来週TODO

## 227. [Basic Calculator II](https://leetcode.com/problems/basic-calculator-ii/)

### Sol1. traverse with stack 

TODO

### Sol2. traversion without stack

TODO

### Sol3. Iterator with stack: O(N) time, O(N) space

### Sol4. Iterator with stack: O(N) time, O(1) space


```py

class Solution:
    def calculate(self, s: str) -> int:
        itr = StringIterator(s)
        ret = 0
        last_operation = None
        
        while itr.has_next():
            is_muliplication_or_division, curr_ope = itr.next()
            if is_muliplication_or_division:
                last_operation = lambda x, last_operation=last_operation, curr_ope=curr_ope: x+curr_ope(last_operation(0))
            else:
                if last_operation is not None:
                    ret += last_operation(0)
                last_operation = curr_ope
        return ret + last_operation(0)
  
        
class StringIterator:
    def __init__(self, string):
        # 一つ目の数字を特別扱いせずにnext()で取れるようにした！
        # 我ながらよく思いついた！
        if string[0] != '-':                                  
            self.string = ('+' + string).replace(' ', '')     # ここで空文字を抜くのを最初忘れていた
        self.curr_idx = -1
        
        
    def has_next(self):
        return self.curr_idx + 1 <= len(self.string) - 1
    
    def next(self):
        self.curr_idx += 1
        operation_str = self.string[self.curr_idx]
        temp = self.curr_idx + 1
        while (self.curr_idx + 1 < len(self.string)) and self.string[self.curr_idx + 1].isdigit():
            self.curr_idx += 1
        num = int(self.string[temp:self.curr_idx+1])

        operation_to_lambda = {
            '+': lambda x, num=num: x + num,
            '-': lambda x, num=num: x - num,
            '*': lambda x, num=num: x * num,
            '/': lambda x, num=num: x // num if x > 0 else -((-x)//num),  # 負の場合の条件分岐を最初考えてなかった
        }
        
        return (operation_str in ['*', '/']), operation_to_lambda[operand_str]
```

### 所感

自力でsol3を思いついた。その後、模範解答を見て、確かにstackでO(N)使う必要はないことに気が付き、sol4に書き直した。


- Iteratorを使ったら綺麗かなーと思って使ったところ、iteratorに処理を押し込めれたのは確かに綺麗だが、代わりにラムダ関数が発生してそこが辛くなった。ラムダ関数の挙動をちゃんと理解していなくて、30分くらい詰まった。https://twitter.com/kdjp20/status/1426855305032339460
- `operation_to_lambda['/']` の負の場合の条件分岐を入れていなかった。
- 最初に与えられたstringから空文字を抜くのを忘れていた。
- operation(演算子+,-,,)をoperand(被演算子)と命名していたw



これに関連して、いおりんさんと焼肉ランチに向かう途中に、逆ポーランド法(reverse polish)を教えてもらった。「reverse polishを作る ->
 それをstackで計算」により、シンプルに実装できるらしい。(実装はしてない)

## [146. LRU Cache](https://leetcode.com/problems/lru-cache/)



### 所感

220の解法2を思い出すついでにsortedset, orderedset周りに関してまとめた。 https://twitter.com/kdjp20/status/1426808248246497282 ので、ordered set繋がりで復習する。当然だが、ordered setを使う実装では足りない。doubly linked listで実装できるようにすべき。

言わずもがなの超重要問題。

### 解法

```py
class Node:
    def __init__(self):
        self.key = None
        self.val = None
        self.next = None
        self.prev = None

        
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.key_to_node = dict()
        self.head = Node() # pseudo node
        self.tail = Node() # pseudo node
        self.head.next = self.tail      # new <-----> old
        self.tail.prev = self.head
        
    def get(self, key):
        if key not in self.key_to_node:
            return -1
        
        node = self.key_to_node[key]
        self.remove_node(node)
        self.add_to_head(node)
        return node.val

        
    def put(self, key, value):
        if key in self.key_to_node:
            node = self.key_to_node[key]
            node.val = value
            self.remove_node(node)
            self.add_to_head(node)
        else:
            new_node = Node()
            new_node.key = key
            new_node.val = value
            self.key_to_node[key] = new_node
            self.add_to_head(new_node)
            
        if len(self.key_to_node) > self.capacity:
            tail_node = self.tail.prev
            self.remove_node(tail_node)
            del self.key_to_node[tail_node.key]
        
    def remove_node(self, node):
        node.prev.next, node.next.prev = node.next, node.prev
```


### 所感

Rizkyに手伝ってもらった。



## 220. [Contains Duplicate III](https://leetcode.com/problems/contains-duplicate-iii/)


### 解法2 SortedSet O(Nlog(min(N, k)))time

```py
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        nums_in_window = SortedSet()
        
        for right in range(len(nums)):
            if k < right:
                nums_in_window.remove(nums[right - k - 1])    # logk
            idx = bisect.bisect_left(nums_in_window, nums[right] - t)     # logk
            if idx <= len(nums_in_window)-1 and nums_in_window[idx] <= nums[right] + t:
                return True
            nums_in_window.add(nums[right])     # logk
            
        return False
```

pythonのstdlibにはsorted setはないので、3rd party libのsortedcontainers.SortedSetを使うか、自前でBalanced BSTを使用してsorted setを実装する必要がある(しんどい)。

### 解法3 Buckets O(N) time


```py
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        # id of the bucket is defined by x // t+1
        # key is id and value is number which is in bucket.
        # Note that multiple numbers cannot be in the same bucket, because in that case,
        # True should have been returned.
        buckets = dict()
        for right in range(len(nums)):
            if k < right:
                del buckets[self.get_bucket_id(t+1, nums[right - k - 1])]
            
            bucket_id = self.get_bucket_id(t+1, nums[right])
            if bucket_id in buckets:
                return True
            if (bucket_id - 1 in buckets) and abs(buckets[bucket_id - 1] - nums[right]) <= t:
                return True
            if (bucket_id + 1 in buckets) and abs(buckets[bucket_id + 1] - nums[right]) <= t:
                return True
            
            buckets[bucket_id] = nums[right]
        return False
            
    def get_bucket_id(self, bucket_size, num):
        return num // bucket_size
```






バケツの大きさはt+1が良い理由は、See: https://twitter.com/kdjp20/status/1426827866558189569





### 所感


この問題、実は五ヶ月前の21/03/14にすでに解いていたが、完全に忘れていた。   
解法2を思い出すついでにsortedset, orderedset周りに関してまとめた。 https://twitter.com/kdjp20/status/1426808248246497282    
解法3天才。[←村人さんのおすすめ](https://twitter.com/ronin_2020/status/1371056641903194113)    

解法2も解法3も、「同一windowには重複する要素は入っていない、 同一バケツ内には複数の要素は入っていない。なぜならもしそんなことが起こっていたらそれらの要素が入る時点ですでにTrueが返されているはずだから」と言うのが面白いし天才的なところ。



## 219. [Contains Duplicate II](https://leetcode.com/problems/contains-duplicate-ii/)

### 所感

~~簡単。~~良問すぎる。以下3つも学んだ。


- space complexiyは、O(min(n, k)) assuming that n = length of a given list。
   - 確かに。脱帽。
- 最初のk個とそれ以降のk個のそれぞれのチェックに関しては、以下のような実装により、一つのループにまとめられる。
   - 確かに。それはそうだ。


```py
class Solution:
    # なお、k=0の場合もちゃんとバグらずに動いてくれる
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        left = 0
        right = 0
        num_to_cnt = defaultdict(int)
        num_to_cnt[nums[0]] = 1
        
        while right + 1 < len(nums):
            if right - left >= k:      ### このif節が肝。
                num_to_cnt[nums[left]] -= 1
                left += 1
            
            if num_to_cnt[nums[right + 1]] > 0:
                return True
            right += 1
            num_to_cnt[nums[right]] += 1
        return False
```

- しかも、この問題、実は要素のcntをキープしなくても良い！！！
    - 例えばleft+=1で除外すべき数字が、slideing windowの範囲内に他にも存在するかもしれないので安易に `del num_set[left]`してはいけないと思っていた。が、もし、他の場所に存在するのならば、その数字にrightが到達した時点でreturn Trueされているはずだったので、ないと考えて良い。天才。
    - とするともはやleftとrightを追う必要もない。
    
```py

class Solution:
    def containsNearbyDuplicate(self, nums: List[int], k: int) -> bool:
        nums_set = set()
        
        for right in range(len(nums)):
            if k < right:
                nums_set.remove(nums[right-k-1])
            if nums[right] in nums_set:
                return True
            nums_set.add(nums[right])
        return False
```

## [204. Count Primes](https://leetcode.com/problems/count-primes/)



### 所感

ただのエラトステネスの典型問題。`less than` nの素数のカウントだったので、ちゃんと問題よめ。
mhttps://leetcode.com/problems/count-primes/solution/

## 1175. [Prime Arrangements](https://leetcode.com/problems/prime-arrangements/)




### 所感

素数判定&素数カウントをちゃんとエラトステネスの篩で実装したのは地味に初めてかもしれない。    
1~Nまでの素数(の数)を求めるには、


```python
        # 最初のFalseは、listのindexとiを合わせるためのpseudo element
        are_primes = [False, False] + [True] * (n-1)
        for num in range(2, math.ceil(n ** 0.5)+1):    # +1 するの忘れてた
            if not are_primes[num]:
                continue
            i = 2
            while num * i <= n:
                are_primes[num * i] = False
                i += 1
# n = 5 の場合、are_primes = [False, False, True, True, False, True]となる。

```

## 307. [Range Sum Query - Mutable](https://leetcode.com/problems/range-sum-query-mutable/)



### 所感

もはやcumulative sum関係ないが、ついでにこれも復習する。これは、cumulative sumを使うと、更新時にO(N)かかってしまうのでだめ。

## [303. Range Sum Query - Immutable](https://leetcode.com/problems/range-sum-query-immutable/)


### 所感

queryがたくさん投げられるのでcumulative sumのリストを作っておくという典型問題だった。

## 53. [Maximum Subarray](https://leetcode.com/problems/maximum-subarray/)



###  cumulative sum  = O(n**2) time


よくよく考えると、cumulative sumを使わなくても以下のようにすればよかった。というかこっちはspaceが定数ですむ。

```py
max_sum = -10000

for i in range(len(nums)):
    sum_nums = 0
    for j in range(i, len(nums)):
        sum_nums += nums[j]
        max_sum = max(max_sum, sum_nums)
return max_sum
```



### DP  = O(n) time

### 所感

560をやっていて、思い出して復習してみた。
cumulative sum(累積和)の復習をするつもりだったが、この問題は、よくよく考えると累積和を使ったらspaceが無駄になる。

多分cumulative sumが活躍するのは、クエリが複数回飛んでくる場合に備えて前処理をするケース。
~~それ以外のケース、特に左端と右端を総当たりするケースでは、計算時間がO(n**2)かかるので、上記のスニペットのように二重ループを回すほうがspaceはsaveできる。~~ いやいやいや、hashmapを使うことで time=O(N), space=O(N)でいける。

two pointersでは解けないが、runnign sumだと解ける問題は See:atcoder(zero sum range) https://atcoder.jp/contests/agc023/tasks/agc023_a

## 560. [Subarray Sum Equals K](https://leetcode.com/problems/subarray-sum-equals-k/)


### 解法１ 「cumulative sums => Counterのhashmap」O(n) 時間、O(1)スペース


- => と書いたが、これone pathで解ける。
- => と書いたが、one pathは結構時間掛かるので2pathでやるほうが安全かも。


(1pathとは、cumulative sum作るのを含めて1path)

```py
    # cumulative sums - 1 path
    def subarraySum(self, nums, k):
        curr_cum_sum = 0
        cum_sum_to_cnt = defaultdict(int)
        cum_sum_to_cnt[0] = 1
        ret = 0
        
        for num in nums:
            curr_cum_sum += num
            ret += cum_sum_to_cnt[curr_cum_sum - k]
            cum_sum_to_cnt[curr_cum_sum] += 1
        return ret
                

    # cumulative sums - 2 path
    def _subarraySum(self, nums: List[int], k: int) -> int:
        cum_sums = [0]
        for num in nums:
            cum_sums.append(cum_sums[-1] + num)
    
        ret = 0
        sum_cnts = defaultdict(int)
        for cum in cum_sums:
            if cum - k in sum_cnts:
                ret += sum_cnts[cum - k]
            sum_cnts[cum] += 1

        return ret
```


### ~~解法２  cumulative sumsをとってからの二重ループ O(n**2)時間、O(n)スペース~~ 忘れてよし

解法３の下位互換。
### 解法３ ~~二重ループ O(n**2)、O(n)スペース~~ 忘れてよし


### 所感

配列に負の数が含まれることでかなり複雑化している。と思ったが、そうでもないのかも。
「Cumulative sumをとってそのあとCounterでhash mapを作成する」という定石に従えば瞬殺だった。

ちなみに、正の数のみのstrictly increasing orderの配列なら、以下のよう。(532の解法2では似たような2pointersの実装で、各if節の中でwhileループを回しているが、二重にしないほうが簡単に書ける気がする)


```python
        left, right = 0, 1
        if cumulative_sums[1] - cumulative_sums[0] == k:
            ans = 1
            
        while True:
            print(left, right)
            if (left == right) or (cumulative_sums[right] - cumulative_sums[left] <= k):
                right += 1
                if right == len(cumulative_sums):
                    return ans

                if (cumulative_sums[right] - cumulative_sums[left] == k):
                    ans += 1
            else:
                left += 1
                if (left != right) and (cumulative_sums[right] - cumulative_sums[left] == k):
                    ans += 1
        return ans
```

## 889. [Construct Binary Tree from Preorder and Postorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-postorder-traversal/) (挫折)


### 所感

- https://leetcode.com/problems/construct-binary-tree-from-preorder-and-postorder-traversal/discuss/161268/C%2B%2BJavaPython-One-Pass-Real-O(N) を読もうとするも理解できず。

## 582. [Kill Process](https://leetcode.com/problems/kill-process/)


### 所感

簡単すぎて解説を読む気になれなかった。
pythonの内包表記は、`{ pid: i for i, pid in enumerate(pid) }`みたいな書き方が可能だということを学んだ。

## [947. Most Stones Removed with Same Row or Column](https://leetcode.com/problems/most-stones-removed-with-same-row-or-column/)


### 所感

UnionFindで、以下忘れてた。

- #union にて、最初に `self.are_in_the_same_group`ならその時点でreturnする
   - これを使わずに自前で実装するのなら、まずfindで親のidxを取得してそれらがイコール化を比較しないといけないのに、引数のxとyを比較していたw
- `union by rank`という名称


## 1048. [Longest String Chain](https://leetcode.com/problems/longest-string-chain/)



### 所感



## 

## 740. [Delete and Earn](https://leetcode.com/problems/delete-and-earn/)


### 所感

最適

## 255. [Verify Preorder Sequence in Binary Search Tree](https://leetcode.com/problems/verify-preorder-sequence-in-binary-search-tree/)

intの配列が与えられる。それが、binary search treeの正しいpreorder traversalした結果であるならTrueを返せ。(全ての要素はuniqueである)

<=> (読みかえ)「preorder traversalの結果があるintの配列になるようなtreeは複数あるが、そのうち一つでもbinary search treeになっている場合はtrueを返せ」


### 解法1 traversals (TLE)

preorderも、inorderも、最初の要素がrootになることには変わりないことに注目する。

よって、[10,2,3,8,12,14,15]は有効。[10,2,3,8,12,7,14]はだめ。
要は、配列の最初の要素のrootの値を境に、前半(left children)と後半(right children)に明確に分け(られ)る、という操作を再起的に全てのleafにたどり着くまでできた場合、その配列はbinary search tree。

まあ、見るからに時間がかかりそうな実装ではある。O(logN) ~ (N**2)time、O(logN) ~ log(N)space がかかる。

```py

class Solution:
    def verifyPreorder(self, preorder: List[int]) -> bool:
        self.preorder = preorder
        return self.is_bst(0, len(preorder)-1)
        
        
    def is_bst(self, left, right):  # range of idx in the original `preorder` list
        # 呼び出し元で[1,100,101,102]のようだった場合、
        # left > rightになっている
        if left >= right:
            return True
        root_val = self.preorder[left]
        first_larger_idx = None
        for i in range(left + 1, right + 1):
            if first_larger_idx is not None and self.prorder[i] < root_val:
                return False
            if first_larger_idx is None and root_val < self.preorder[i]:
                first_larger_idx = i
        if first_larger_idx is None:
            return self.is_bst(left+1, right)
            
        return self.is_bst(left+1, first_larger_idx-1) and self.is_bst(first_larger_idx, right)

```

### 解法2. stack

https://www.youtube.com/watch?v=0kkVobZ6Ebc が鬼のようにわかりやすかった。他に何もいらない。

```py
class Solution:
    def verifyPreorder(self, preorder: List[int]) -> bool:
        stack = []
        max_so_far = -1
        
        for num in preorder:
            while 0 < len(stack) and stack[-1] < num:
                max_so_far = stack.pop()
            if num < max_so_far:
                return False
            stack.append(num)
        return True
```



### 所感

binary searchを使う方向でトータル3時間くらい考えてたけど、これわんちゃん5分でできる問題だな。かなり良問。
解法１のtime complexityを考えるついでに、ソートを復習した。https://twitter.com/kdjp20/status/1427636978980360199





## 284. [Peeking Iterator](https://leetcode.com/problems/peeking-iterator/)


Iteratorをwrapし、peekメソッドを持つ以下のようなiteratorクラスを実装せよ。

```
PeekingIterator(int[] nums) Initializes the object with the given integer array nums.
# int next() Returns the next element in the array and moves the pointer to the next element.
# bool hasNext() Returns true if there are still elements in the array.
# int peek() Returns the next element in the array without moving the pointer
```


### 解法

簡単。next_valというフィールドを持っておくだけ。初期化、nextの呼び出し後には必ずnext_valを必ず埋めておく(つまり、常に一つ先の要素を取り出しておく)、という実装をすることで、めっちゃシンプルに実装できるようになる。

```py
class PeekingIterator:
    def __init__(self, iterator):
        self._next = iterator.next()
        self._iterator = iterator

    def peek(self):
        return self._next

    def next(self):
        ret = self._next
        self._next = self._iterator.next() if self._iterator.hasNext() else None
        return ret
```


### 所感

`常に一つ先の要素を取り出しておく`という発想は思いつかなかった。脱帽。

## 341. [Flatten Nested List Iterator](https://leetcode.com/problems/flatten-nested-list-iterator/)

```
class NestedInteger:
#    def isInteger(self) -> bool:
#    def getInteger(self) -> int:
#    def getList(self) -> [NestedInteger]:   ##### 空配列を返すこともある！！！
```

上記のようなNestedIntegerの配列が渡される時、これをflattenして、順々に値を返すiteratorオブジェクトを実装せよ。実装するべきiteratorのinterfaceは以下。

```
class NestedIterator:
# NestedIterator(List<NestedInteger> nestedList) Initializes the iterator with the nested list nestedList.
# int next() Returns the next integer in the nested list.
# boolean hasNext() Returns true if there are still some integers in the nested list and false otherwise.
```

### 解法1. 初期化時にrecursionでtraverseしてlistを初期化 (思いついた)

まあ、これが一番に思いつく。ただ、iterator patternは、pointerだけを保持していいのでspaceをセーブできるというのが売りなので、そういう意味でこれを実装したら「僕デザインパターン知りません！」と堂々と言い放っているようなものなので注意。

あと、最初実装した時にわざわざrecursive funcからリストを返し、それを呼び出しがわでextendしていくという謎実装をしていた。そんなことしない、かくfunc内で(global変数あるいは)インスタンス変数のリストにappendしていく。


### 解法2. 初期化時にstackを使用してtraverseしてlistを初期化(思いついた)

解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。
これも内部でまるっとデータ構造持っておかないといけないのでいい解法とは言えない。



### 解法3. stackを利用、#next()が呼ばれる度に次の要素までtraverse

解法2とアイデアは同じ。解法1のrecursionの代わりにstackを用いただけ。最初にNestedIntegerの配列をリストに逆順につめ、これをstackとして用いる。NestedIntegerはポインタでしかないので、spaceをsaveできる。ただし、与えられたリストの要素が全てintを表すNestedIntegerなら、解法2とspace complexityは同じという認識。


引っかかった点として、NestInteger.getList()は空配列になりうるというところ。したがって、`hasNext(), next()`が呼ばれた時点でtailの要素がlistのNestedIntegerだった場合、それを分解してやる(処理を繰り返す)必要がある。しかも、このロジックはhasNext()とnext()の両方で使用されるので、`def make_stack_top_an_integer(self)`としてメソッドに切り出すのが吉。

```
class NestedIterator:
    def __init__(self, nestedList: [NestedInteger]):
        self.stack = list(reversed(nestedList))
    
    def next(self) -> int:
        self.make_stack_top_an_integer()
        return self.stack.pop().getInteger()
    
    def hasNext(self) -> bool:
        self.make_stack_top_an_integer()
        return len(self.stack) > 0
```

### 解法4. Using a Generator

読み流した。yieldとかのお話。



### 所感

あとこの問題、「もう残りの要素がない状態で#next()が呼ばれたらどうしましょう？？？」って聞くと「こいつ優秀！」って思われるらしい。

解くこと自体は簡単。計算時間求めるのはややこい。
解答後、iterator patternについてまとめた。
https://github.com/kudojp/RoadToSuperCoder/blob/master/design_patterns.ipynb

振り返ってみるとかなり良問な気がしてきた。

## 117. [Populating Next Right Pointers in Each Node II](https://leetcode.com/problems/populating-next-right-pointers-in-each-node-ii/)

binary searchが与えられる。各nodeのnextフィールドに、その右隣のnodeを指すpointerを代入せよ。O(1)spaceで。


### 解法1 O(N) space

まあ、普通にqに同階層のnodeを詰めるBFSで実装する。


### 解法2 待望のO(1) space

BFS。現在traverseしているより一段下のlevelのnodesのnextフィールドを設定することで、qを使用しなくてもその階層の次のnodeをたどっていける。(一段下というのが肝)

```py

## 最初にゴニョゴニョ

while leftmost is not None:
    curr = leftmost
    prev = None
    leftmost = None
    while curr is not None:
        for child in [curr.left, curr.right]:
            if child is None:
                continue
            if prev is None:    # when it is a first node in the level
                leftmost = child
                prev = child
                continue
            prev.next = child
            prev = prev.next
        curr = curr.next
```


### 所感

本当にびっくりするくらい美しい良問だと思う。脱帽。




## 532. [K-diff Pairs in an Array](https://leetcode.com/problems/k-diff-pairs-in-an-array/)

int[]が与えられる時、差がkとなるような数字のユニークな組み合わせは何通りあるか？

### O(N)時間: counterのhashを作成後にkeyをループする

```py
val_to_cnt = Counters(nums)
checked = set()
cnt = 0

for val in val_to_cnt:
    if val in checked:
        continue
    if val + k in val_cnt:  # +kの場合のみ考えることでsimplifyした。
        cnt += 1
return cnt
```

上では書いてないが、k = 0の場合が見事なエッジケースw。

### O(NlogN)時間: ソートしてから2pointers

実装はややこい。書き終わってみたらまあなるほどって感じ。
気が向いたらかく　TODO

### 所感

解法２は、spaceは解法１と同じO(N)、timeは解法１より遅い上に、解法１より実装がややこしいので、まあ、解法１を使うでしょ。



[参考] ソートのtimecomplexityは `O(logN)` ~ `O(N)`かかる。
(meege sortがO(N), quick sortがO(logN)~O(N))

- https://twitter.com/kdjp20/status/1389091424063213568
- https://twitter.com/kdjp20/status/1389817538062680066


## 1648. [Sell Diminishing-Valued Colored Balls](https://leetcode.com/problems/sell-diminishing-valued-colored-balls/) (挫折)

## 1539. [Kth Missing Positive Number](https://leetcode.com/problems/kth-missing-positive-number/)


strictly increasing orderのintのlistが与えられる。skipされた数字のうちk番目のものを求めよ。(e.g. [1,2,4,6], k=2 -> 5)

### 解法1 (できた)

前からループを回す。計算時間O(N)

### 解法２ (できた)

binary searchを使う。
配列のあるindexまでにどれだけ数字がskipされたかは、arr[idx] - idx で求められることを利用。


bisect_left的実装でidxを見つけた後、そのidxからbackwardに戻っていくのではなく、idx - 1からforwardに探索してやるとより直感的。

### 所感



いい binary searchの練習になった。解法2で、一つidxを戻してforwardにたどっていくのわかりやすいなーって思った。

## 875. [Koko Eating Bananas](https://leetcode.com/problems/koko-eating-bananas/)

複数のバナナの山(`int[]`)をh時間内に食べ切ることのできる最も遅いスピードを求めよ。ただし、同じ時間フレーム(1時間)以内に複数のバナナに手をつけることはできない。

### 解法1 (heapを使用、思いついた -> tle) 

計算時間がO(h logN)かかってしまいTLE。    
TODO: リンク貼る

### 解法2 (binary searchを使用し、O(N * log max_pile_size)

計算時間を`O(N * log max_pile_size)`まで縮められた。   
(bisect_leftの実装にミスって116/118でTLEを連発してしまった。Javaで書き直したが、そういう問題ではなかったw)    


```python    
low = math.ceil(sum(piles) / h)
high = max(piles)

# イメージは、[f,f,f,f,t,t,t,t,t](x)の中で、一番左のtのindexを取得する。
# これが一番の肝。helper functionでtaking_timeなんかを取ってhと比較したりするとややこしい。

while low < high:
    mid = (low + high) // 2
    is_possible = is_possible(mid)
    
    if is_possible: # speed must be slower than mid
        high = mid
    else: # h <= t
        low = mid + 1 # speed is higher or 
    return low     # low != len(piles)
```
        
        
### 所感

解法2はグッチさんとマックで1時間くらい考えてやっと思いついた。
helper_functionをうまく設定できるかが鍵だと思う。    
bisect_leftに関してまとめた。 https://twitter.com/kdjp20/status/1424250217268871170

## 300.  [Longest Increasing Subsequence](https://leetcode.com/problems/longest-increasing-subsequence/)

intの配列において、最長のLISの配列の長さを求める問題。O(NlogN)時間でとけ。
(ex) [0,3,1,6,2,2,7] → [0,1,2,7] なので、ans = 4

### 解法1 (似たようなの思いついた)

与えられた配列の各要素に対応し、その要素までのLISの配列長を表すリストdpを作成していく。
初期条件は、dp[0] = 1。
dp[1:]に関しては、max(それまでに登場した要素で、現在の要素の値未満の要素のLIS長)+1。なお、そこまでにその値未満の要素が存在しない場合には1を入れる。

### 解法2 (全く思いつかなかった)

dpリストを作成し、これをupdateしていく。初期値は、dp=[nums[0]]。与えられた配列nums[1:]の要素を順々に見ていき、その要素がdpのどの要素よりも大きい場合には、それをdpの末尾にappendする。そうでない場合には、dpの中でそれ以上の値でindexが最も若い要素とそれを入れ替える。最終的に完成したlen(dp)が、求める値である。


```
着眼点としては、[1, 5, 3]が与えられた時、「３が加わる瞬間に５は用無し」になること。
注意点としては、「最終的に得られるdpリストの長さは、LSIの最大長に等しい」が、「それらの要素がLSIを作る要素を表しているわけではない」ということ。例えば、[1,3,5,2]が与えられた時、dpは、[1]>[1,3]>[1,3,5]>[2,3,5]となるが実際にLSIを作るのは[1,3,5]。
```

なお、「dpの中でそれ以上の値の一つ目の要素」を発見する過程でbinary searchを用いることでO(NlogN)時間になる。



### 所感

解法２が天才的。どうやったら思いつくんだ、、、
bisect_leftを使ったが、特に場合分けが要らなかった。これは、bisect_left(list, x)がlen(list)を返すケースを考えなくて良いから。


## 75. [Sort Colors](https://leetcode.com/problems/sort-colors/)

`Dutch national flag problem`   
0,1,2のいずれかの要素からなる配列(e.g. [1,2,2,1,0])を0, 1, 2のセクションに分かれるように、inplaceに入れ替えろ。

### 解法1 (my solution)

- １巡目に、0,1,2のそれぞれの出現数cnt0, cnt1, cnt2を数える。
- ２巡目に、(left, right) = (0, cnt1)から初めて、先頭のcnt0個の要素が0になるように(1or2 <-> 0)のswapを繰り返す。
- ３巡目に、(left, right) = (cnt0, cnt0+cnt1)から初めて、真ん中のcnt1個の要素が1になるように(1 <-> 2)のswapを繰り返す。


### 解法2 (似たようなの思いついたけど、edge caseをがややこそうだったので撤退)

次に、0, 2を置かれるべきindexをそれぞれ(p0, p1) = (0, len(nums)-1)に、またcurr = 0を定義する。


```python
while curr <= p1:   # =が必要らしい
    if curr == 0:
        (p0とcurrをswap)
        p0 += 1
        curr += 1       # currも右に動かすらしい。これは、currがp0よりも後ろにいるのを防ぐため??
    elif curr == 1:
        curr += 1
        # p1とのswap後には、currの箇所に２が来るかもしれないからまだcurrを右に動かすことはできない
    else:
        (p1とcurrをswap)
```

### 所感

解法１ではエッジケースを考え漏れていた。(cnt0=len(nums)つまり0のみの配列の場合や、cnt0+cnt1=len(nums)つまり0と1のみの配列の場合)
解法２は意外にもエッジケースを考える必要はなかった。ただし、コメントに書いた箇所の考慮がまたしっくりきていない。
